In [4]:
# Plain Minimax
import math, time, copy

# === Color Utility ===
class Color:
    MAX = "\033[92m"      # Green for AI (Maximizer)
    MIN = "\033[91m"      # Red for AI (Minimizer)
    TITLE = "\033[95m"    # Purple for titles
    STAT = "\033[97m"     # White for stats
    RESET = "\033[0m"     # Reset color

# === Global Counters ===
stats = {"nodes": 0, "max_depth": 0}

# === Game Logic ===
def print_board(board):
    print(Color.STAT + "\nCurrent Board:" + Color.RESET)
    for row in board:
        print(" | ".join([cell if cell != "" else " " for cell in row]))
    print("-" * 9)

def check_winner(board):
    for i in range(3):
        if board[i][0] == board[i][1] == board[i][2] != "":
            return board[i][0]
        if board[0][i] == board[1][i] == board[2][i] != "":
            return board[0][i]
    if board[0][0] == board[1][1] == board[2][2] != "":
        return board[0][0]
    if board[0][2] == board[1][1] == board[2][0] != "":
        return board[0][2]
    return None

def moves_left(board):
    return any("" in row for row in board)

def evaluate(board):
    winner = check_winner(board)
    if winner == "X":
        return 10
    elif winner == "O":
        return -10
    return 0

# === Plain Minimax (No Pruning) ===
def minimax_no_pruning(board, depth, is_maximizing):
    stats["nodes"] += 1
    stats["max_depth"] = max(stats["max_depth"], depth)

    score = evaluate(board)
    if score != 0 or not moves_left(board):
        return score

    if is_maximizing:
        best = -math.inf
        for i in range(3):
            for j in range(3):
                if board[i][j] == "":
                    board[i][j] = "X"
                    val = minimax_no_pruning(board, depth + 1, False)
                    board[i][j] = ""
                    best = max(best, val)
        return best
    else:
        best = math.inf
        for i in range(3):
            for j in range(3):
                if board[i][j] == "":
                    board[i][j] = "O"
                    val = minimax_no_pruning(board, depth + 1, True)
                    board[i][j] = ""
                    best = min(best, val)
        return best

# === AI Move Function ===
def best_move_minimax(board, is_maximizing):
    start = time.perf_counter()
    stats.update({"nodes": 0, "max_depth": 0})

    best_val = -math.inf if is_maximizing else math.inf
    best_move = None

    for i in range(3):
        for j in range(3):
            if board[i][j] == "":
                board[i][j] = "X" if is_maximizing else "O"
                val = minimax_no_pruning(board, 0, not is_maximizing)
                board[i][j] = ""
                if is_maximizing and val > best_val:
                    best_val = val
                    best_move = (i, j)
                elif not is_maximizing and val < best_val:
                    best_val = val
                    best_move = (i, j)

    end = time.perf_counter()

    # Summary output
    player = "MAX (X)" if is_maximizing else "MIN (O)"
    print(f"\n{Color.TITLE}=== {player} Move Summary ==={Color.RESET}")
    print(Color.STAT + f"Chosen Move: {best_move}")
    print(f"Evaluation Score: {best_val}")
    print(f"Nodes Explored: {stats['nodes']}")
    print(f"Max Depth Reached: {stats['max_depth']}")
    print(f"Execution Time: {(end - start):.5f} seconds" + Color.RESET)
    print(Color.TITLE + "-----------------------------------" + Color.RESET)

    return best_move

# === Auto Game Loop (AI vs AI using Minimax) ===
def autoplay_minimax():
    board = [["", "", ""], ["", "", ""], ["", "", ""]]
    print(Color.TITLE + "\n🤖 AUTO-PLAY: Plain Minimax Simulation (Both AIs)" + Color.RESET)
    print_board(board)

    current_player = True  # True = Maximizer (X), False = Minimizer (O)

    while True:
        move = best_move_minimax(board, current_player)
        if move:
            board[move[0]][move[1]] = "X" if current_player else "O"

        print_board(board)

        winner = check_winner(board)
        if winner:
            print(Color.MAX + f"\n🏆 Winner: {winner} (via Plain Minimax)" + Color.RESET)
            break
        if not moves_left(board):
            print(Color.STAT + "\nIt's a draw!" + Color.RESET)
            break

        current_player = not current_player  # Switch turns

# Run the Auto Simulation
if __name__ == "__main__":
    autoplay_minimax()



🤖 AUTO-PLAY: Plain Minimax Simulation (Both AIs)

Current Board:
  |   |  
  |   |  
  |   |  
---------

=== MAX (X) Move Summary ===
Chosen Move: (0, 0)
Evaluation Score: 0
Nodes Explored: 549945
Max Depth Reached: 8
Execution Time: 3.47635 seconds
-----------------------------------

Current Board:
X |   |  
  |   |  
  |   |  
---------

=== MIN (O) Move Summary ===
Chosen Move: (1, 1)
Evaluation Score: 0
Nodes Explored: 59704
Max Depth Reached: 7
Execution Time: 0.33821 seconds
-----------------------------------

Current Board:
X |   |  
  | O |  
  |   |  
---------

=== MAX (X) Move Summary ===
Chosen Move: (0, 1)
Evaluation Score: 0
Nodes Explored: 7331
Max Depth Reached: 6
Execution Time: 0.04237 seconds
-----------------------------------

Current Board:
X | X |  
  | O |  
  |   |  
---------

=== MIN (O) Move Summary ===
Chosen Move: (0, 2)
Evaluation Score: 0
Nodes Explored: 934
Max Depth Reached: 5
Execution Time: 0.00495 seconds
-----------------------------------

Cur

In [5]:
# With Alpha Beta pruning
import math, time

# === Color Utility ===
class Color:
    MAX = "\033[92m"      # Green for AI (Maximizer)
    MIN = "\033[91m"      # Red for AI (Minimizer)
    TITLE = "\033[95m"    # Purple for titles
    STAT = "\033[97m"     # White for stats
    RESET = "\033[0m"     # Reset color

# === Global Stats ===
stats = {"nodes": 0, "pruned": 0, "max_depth": 0}

# === Game Logic ===
def print_board(board):
    print(Color.STAT + "\nCurrent Board:" + Color.RESET)
    for row in board:
        print(" | ".join([cell if cell != "" else " " for cell in row]))
    print("-" * 9)

def check_winner(board):
    for i in range(3):
        if board[i][0] == board[i][1] == board[i][2] != "":
            return board[i][0]
        if board[0][i] == board[1][i] == board[2][i] != "":
            return board[0][i]
    if board[0][0] == board[1][1] == board[2][2] != "":
        return board[0][0]
    if board[0][2] == board[1][1] == board[2][0] != "":
        return board[0][2]
    return None

def moves_left(board):
    return any("" in row for row in board)

def evaluate(board):
    winner = check_winner(board)
    if winner == "X": return 10
    elif winner == "O": return -10
    return 0

# === Minimax with Alpha–Beta Pruning ===
def minimax_ab(board, depth, alpha, beta, is_maximizing):
    stats["nodes"] += 1
    stats["max_depth"] = max(stats["max_depth"], depth)
    score = evaluate(board)
    if score != 0 or not moves_left(board):
        return score

    if is_maximizing:
        max_eval = -math.inf
        for i in range(3):
            for j in range(3):
                if board[i][j] == "":
                    board[i][j] = "X"
                    eval_val = minimax_ab(board, depth + 1, alpha, beta, False)
                    board[i][j] = ""
                    max_eval = max(max_eval, eval_val)
                    alpha = max(alpha, eval_val)
                    if beta <= alpha:
                        stats["pruned"] += 1
                        return max_eval
        return max_eval
    else:
        min_eval = math.inf
        for i in range(3):
            for j in range(3):
                if board[i][j] == "":
                    board[i][j] = "O"
                    eval_val = minimax_ab(board, depth + 1, alpha, beta, True)
                    board[i][j] = ""
                    min_eval = min(min_eval, eval_val)
                    beta = min(beta, eval_val)
                    if beta <= alpha:
                        stats["pruned"] += 1
                        return min_eval
        return min_eval

# === AI Move with Alpha–Beta ===
def best_move_ab(board, is_maximizing):
    start = time.perf_counter()
    stats.update({"nodes": 0, "pruned": 0, "max_depth": 0})

    best_val = -math.inf if is_maximizing else math.inf
    best_move = None

    for i in range(3):
        for j in range(3):
            if board[i][j] == "":
                board[i][j] = "X" if is_maximizing else "O"
                val = minimax_ab(board, 0, -math.inf, math.inf, not is_maximizing)
                board[i][j] = ""
                if is_maximizing and val > best_val:
                    best_val, best_move = val, (i, j)
                elif not is_maximizing and val < best_val:
                    best_val, best_move = val, (i, j)

    end = time.perf_counter()
    player = "MAX (X)" if is_maximizing else "MIN (O)"

    # === Summary Output ===
    print(f"\n{Color.TITLE}=== {player} Move Summary ==={Color.RESET}")
    print(Color.STAT + f"Chosen Move: {best_move}")
    print(f"Evaluation Score: {best_val}")
    print(f"Nodes Explored: {stats['nodes']}")
    print(f"Branches Pruned: {stats['pruned']}")
    print(f"Max Depth Reached: {stats['max_depth']}")
    print(f"Execution Time: {(end - start):.5f} seconds" + Color.RESET)
    print(Color.TITLE + "-----------------------------------" + Color.RESET)
    return best_move

# === Auto Game Loop (AI vs AI using Alpha–Beta) ===
def autoplay_alpha_beta():
    board = [["", "", ""], ["", "", ""], ["", "", ""]]
    print(Color.TITLE + "\n🤖 AUTO-PLAY: Minimax with Alpha–Beta Pruning (Both AIs)" + Color.RESET)
    print_board(board)

    current_player = True  # True = Maximizer (X), False = Minimizer (O)

    while True:
        move = best_move_ab(board, current_player)
        if move:
            board[move[0]][move[1]] = "X" if current_player else "O"

        print_board(board)

        winner = check_winner(board)
        if winner:
            print(Color.MAX + f"\n🏆 Winner: {winner} (via Alpha–Beta Pruning)" + Color.RESET)
            break
        if not moves_left(board):
            print(Color.STAT + "\nIt's a draw!" + Color.RESET)
            break

        current_player = not current_player

# Run Auto-Play Simulation
if __name__ == "__main__":
    autoplay_alpha_beta()




🤖 AUTO-PLAY: Minimax with Alpha–Beta Pruning (Both AIs)

Current Board:
  |   |  
  |   |  
  |   |  
---------

=== MAX (X) Move Summary ===
Chosen Move: (0, 0)
Evaluation Score: 0
Nodes Explored: 30709
Branches Pruned: 13640
Max Depth Reached: 8
Execution Time: 0.17975 seconds
-----------------------------------

Current Board:
X |   |  
  |   |  
  |   |  
---------

=== MIN (O) Move Summary ===
Chosen Move: (1, 1)
Evaluation Score: 0
Nodes Explored: 4089
Branches Pruned: 1547
Max Depth Reached: 7
Execution Time: 0.02076 seconds
-----------------------------------

Current Board:
X |   |  
  | O |  
  |   |  
---------

=== MAX (X) Move Summary ===
Chosen Move: (0, 1)
Evaluation Score: 0
Nodes Explored: 1519
Branches Pruned: 591
Max Depth Reached: 6
Execution Time: 0.00808 seconds
-----------------------------------

Current Board:
X | X |  
  | O |  
  |   |  
---------

=== MIN (O) Move Summary ===
Chosen Move: (0, 2)
Evaluation Score: 0
Nodes Explored: 220
Branches Pruned: 86
Ma

In [1]:
# Human versus AI
import math

# === Color Utility ===
class Color:
    MAX = "\033[92m"      # Green for AI (Maximizer)
    MIN = "\033[91m"      # Red for Human (Minimizer)
    PRUNE = "\033[93m"    # Yellow for Pruning
    MOVE = "\033[96m"     # Cyan for Moves
    BOARD = "\033[94m"    # Blue for board display
    TITLE = "\033[95m"    # Purple for titles
    STAT = "\033[97m"     # White for stats
    RESET = "\033[0m"     # Reset color

# === Global Counters ===
stats = {"nodes": 0, "pruned": 0, "max_depth": 0}

# === Game Logic ===
def print_board(board):
    print(Color.BOARD + "\nCurrent Board:" + Color.RESET)
    for row in board:
        print(" | ".join([cell if cell != "" else " " for cell in row]))
    print("-" * 9)

def check_winner(board):
    for i in range(3):
        if board[i][0] == board[i][1] == board[i][2] != "":
            return board[i][0]
        if board[0][i] == board[1][i] == board[2][i] != "":
            return board[0][i]
    if board[0][0] == board[1][1] == board[2][2] != "":
        return board[0][0]
    if board[0][2] == board[1][1] == board[2][0] != "":
        return board[0][2]
    return None

def moves_left(board):
    return any("" in row for row in board)

def evaluate(board):
    winner = check_winner(board)
    if winner == "X":
        return 10
    elif winner == "O":
        return -10
    return 0

# === Minimax + Alpha–Beta Pruning ===
def minimax(board, depth, alpha, beta, is_maximizing):
    stats["nodes"] += 1
    stats["max_depth"] = max(stats["max_depth"], depth)

    score = evaluate(board)
    if score != 0 or not moves_left(board):
        return score

    if is_maximizing:
        max_eval = -math.inf
        print(f"{'  ' * depth}{Color.MAX}MAX turn (depth={depth}){Color.RESET}")
        for i in range(3):
            for j in range(3):
                if board[i][j] == "":
                    board[i][j] = "X"
                    print(f"{'  ' * depth}{Color.MOVE}Trying move X at ({i},{j}){Color.RESET}")
                    eval = minimax(board, depth + 1, alpha, beta, False)
                    print(f"{'  ' * depth}{Color.MAX}Move ({i},{j}) returns score {eval}{Color.RESET}")
                    board[i][j] = ""
                    max_eval = max(max_eval, eval)
                    alpha = max(alpha, eval)
                    if beta <= alpha:
                        stats["pruned"] += 1
                        print(f"{'  ' * depth}{Color.PRUNE}PRUNED at ({i},{j}) α={alpha:.1f}, β={beta:.1f}{Color.RESET}")
                        break
        return max_eval
    else:
        min_eval = math.inf
        print(f"{'  ' * depth}{Color.MIN}MIN turn (depth={depth}){Color.RESET}")
        for i in range(3):
            for j in range(3):
                if board[i][j] == "":
                    board[i][j] = "O"
                    print(f"{'  ' * depth}{Color.MOVE}Trying move O at ({i},{j}){Color.RESET}")
                    eval = minimax(board, depth + 1, alpha, beta, True)
                    print(f"{'  ' * depth}{Color.MIN}Move ({i},{j}) returns score {eval}{Color.RESET}")
                    board[i][j] = ""
                    min_eval = min(min_eval, eval)
                    beta = min(beta, eval)
                    if beta <= alpha:
                        stats["pruned"] += 1
                        print(f"{'  ' * depth}{Color.PRUNE}PRUNED at ({i},{j}) α={alpha:.1f}, β={beta:.1f}{Color.RESET}")
                        break
        return min_eval

# === AI Turn ===
def best_move(board):
    best_val = -math.inf
    move = (-1, -1)
    print(f"\n{Color.TITLE}=== AI Computing Best Move (X) ==={Color.RESET}\n")

    # Reset metrics before each AI decision
    stats.update({"nodes": 0, "pruned": 0, "max_depth": 0})

    for i in range(3):
        for j in range(3):
            if board[i][j] == "":
                board[i][j] = "X"
                print(f"{Color.MOVE}Testing move ({i},{j}){Color.RESET}")
                move_val = minimax(board, 0, -math.inf, math.inf, False)
                print(f"{Color.MAX}Result for move ({i},{j}) = {move_val}{Color.RESET}\n")
                board[i][j] = ""
                if move_val > best_val:
                    best_val = move_val
                    move = (i, j)

    print(f"{Color.MAX}AI chooses move {move} with score {best_val}{Color.RESET}\n")

    # === Emphasized Performance Summary ===
    print(Color.TITLE + "----- PERFORMANCE SUMMARY -----" + Color.RESET)
    print(Color.STAT + f"Nodes explored: {stats['nodes']}")
    print(f"Branches pruned: {stats['pruned']}")
    print(f"Max depth reached: {stats['max_depth']}" + Color.RESET)
    print(Color.TITLE + "--------------------------------" + Color.RESET)

    return move

# === Human Turn ===
def player_move(board):
    while True:
        try:
            row = int(input("Enter row (0-2): "))
            col = int(input("Enter column (0-2): "))
            if 0 <= row <= 2 and 0 <= col <= 2 and board[row][col] == "":
                board[row][col] = "O"
                break
            else:
                print("Invalid move! Try again.")
        except ValueError:
            print("Please enter numeric values between 0 and 2.")

# === Game Controller ===
def play_game():
    board = [["", "", ""], ["", "", ""], ["", "", ""]]
    print(Color.TITLE + "\n🎯 Welcome to Tic Tac Toe (You = O, AI = X)" + Color.RESET)
    print(Color.STAT + "\n🧠 Minimax explores all possible outcomes.")
    print("✨ Alpha–Beta Pruning skips unnecessary paths — watch how node count drops!" + Color.RESET)
    print_board(board)

    while True:
        # Human Move
        player_move(board)
        print_board(board)
        if check_winner(board) == "O":
            print(Color.MIN + "🎉 You win! (MIN)" + Color.RESET)
            break
        if not moves_left(board):
            print("It's a draw!")
            break

        # AI Move
        best = best_move(board)
        board[best[0]][best[1]] = "X"
        print_board(board)

        if check_winner(board) == "X":
            print(Color.MAX + "🤖 AI wins! (MAX)" + Color.RESET)
            break
        if not moves_left(board):
            print("It's a draw!")
            break

# Run the Game
if __name__ == "__main__":
    play_game()



🎯 Welcome to Tic Tac Toe (You = O, AI = X)

🧠 Minimax explores all possible outcomes.
✨ Alpha–Beta Pruning skips unnecessary paths — watch how node count drops!

Current Board:
  |   |  
  |   |  
  |   |  
---------
Enter row (0-2): 0
Enter column (0-2): 0

Current Board:
O |   |  
  |   |  
  |   |  
---------

=== AI Computing Best Move (X) ===

Testing move (0,1)
MIN turn (depth=0)
Trying move O at (0,2)
  MAX turn (depth=1)
  Trying move X at (1,0)
    MIN turn (depth=2)
    Trying move O at (1,1)
      MAX turn (depth=3)
      Trying move X at (1,2)
        MIN turn (depth=4)
        Trying move O at (2,0)
        Move (2,0) returns score -10
        Trying move O at (2,1)
          MAX turn (depth=5)
          Trying move X at (2,0)
            MIN turn (depth=6)
            Trying move O at (2,2)
            Move (2,2) returns score -10
          Move (2,0) returns score -10
          PRUNED at (2,0) α=-10.0, β=-10.0
        Move (2,1) returns score -10
        Trying move O a

In [3]:
# Node Comparison – Minimax vs Alpha–Beta Pruning
import math
import copy
import time
import sys

def minimax_no_pruning(board, depth, is_maximizing):
    """Classic Minimax — no pruning"""
    global nodes_plain
    nodes_plain += 1

    score = evaluate(board)
    if score != 0 or not moves_left(board):
        return score

    if is_maximizing:
        best = -math.inf
        for i in range(3):
            for j in range(3):
                if board[i][j] == "":
                    board[i][j] = "X"
                    val = minimax_no_pruning(board, depth + 1, False)
                    board[i][j] = ""
                    best = max(best, val)
        return best
    else:
        best = math.inf
        for i in range(3):
            for j in range(3):
                if board[i][j] == "":
                    board[i][j] = "O"
                    val = minimax_no_pruning(board, depth + 1, True)
                    board[i][j] = ""
                    best = min(best, val)
        return best


def minimax_with_pruning(board, depth, alpha, beta, is_maximizing):
    """Alpha–Beta Pruning version"""
    global nodes_pruned, prunes_count
    nodes_pruned += 1

    score = evaluate(board)
    if score != 0 or not moves_left(board):
        return score

    if is_maximizing:
        max_eval = -math.inf
        for i in range(3):
            for j in range(3):
                if board[i][j] == "":
                    board[i][j] = "X"
                    eval_val = minimax_with_pruning(board, depth + 1, alpha, beta, False)
                    board[i][j] = ""
                    max_eval = max(max_eval, eval_val)
                    alpha = max(alpha, eval_val)
                    if beta <= alpha:
                        prunes_count += 1
                        return max_eval
        return max_eval
    else:
        min_eval = math.inf
        for i in range(3):
            for j in range(3):
                if board[i][j] == "":
                    board[i][j] = "O"
                    eval_val = minimax_with_pruning(board, depth + 1, alpha, beta, True)
                    board[i][j] = ""
                    min_eval = min(min_eval, eval_val)
                    beta = min(beta, eval_val)
                    if beta <= alpha:
                        prunes_count += 1
                        return min_eval
        return min_eval


# === Compare ===
board = [["X", "O", ""],
         ["", "O", ""],
         ["", "", ""]]

nodes_plain = 0
nodes_pruned = 0
prunes_count = 0

# --- Time both versions ---
start_plain = time.perf_counter()
minimax_no_pruning(copy.deepcopy(board), 0, True)
end_plain = time.perf_counter()

start_prune = time.perf_counter()
minimax_with_pruning(copy.deepcopy(board), 0, -math.inf, math.inf, True)
end_prune = time.perf_counter()

# --- Stats ---
time_plain = end_plain - start_plain
time_prune = end_prune - start_prune
reduction = (1 - (nodes_pruned / nodes_plain)) * 100

# --- Memory (approx) ---
size_plain = sys.getsizeof(nodes_plain) + sys.getsizeof(board) * nodes_plain
size_pruned = sys.getsizeof(nodes_pruned) + sys.getsizeof(board) * nodes_pruned

# === Results Display ===
print(f"\n\033[95m=== Node Comparison ===\033[0m")
print(f"\033[97mPlain Minimax explored:\033[92m {nodes_plain} nodes")
print(f"\033[97mAlpha–Beta Pruning explored:\033[92m {nodes_pruned} nodes")
print(f"\033[97mBranches pruned:\033[91m {prunes_count}")
print(f"\033[97mReduction:\033[93m {reduction:.1f}% fewer nodes evaluated!\033[0m\n")

print(f"\033[95m=== Performance Analysis ===\033[0m")
print(f"\033[97mExecution Time (no pruning):\033[96m {time_plain:.5f} seconds")
print(f"\033[97mExecution Time (with pruning):\033[96m {time_prune:.5f} seconds")
print(f"\033[97mSpeedup:\033[93m {(time_plain / time_prune):.2f}x faster!\033[0m\n")

print(f"\033[95m=== Theoretical Complexity ===\033[0m")
print(f"\033[97mPlain Minimax Time Complexity:\033[92m O(b^d)")
print(f"\033[97mAlpha–Beta Pruning Time Complexity:\033[92m O(b^(d/2)) (in best case)")
print(f"\033[97mSpace Complexity (both):\033[92m O(b * d) (due to recursion stack)\033[0m\n")

print(f"\033[95m=== Memory Approximation ===\033[0m")
print(f"\033[97mEstimated memory (no pruning):\033[96m ~{size_plain/1024:.2f} KB")
print(f"\033[97mEstimated memory (with pruning):\033[96m ~{size_pruned/1024:.2f} KB\033[0m\n")

print(f"\033[92m✅ Observation:\033[0m Alpha–Beta Pruning explores far fewer nodes, executes faster, and uses less memory — while yielding the same optimal move!")



=== Node Comparison ===
Plain Minimax explored: 1055 nodes
Alpha–Beta Pruning explored: 269 nodes
Branches pruned: 96
Reduction: 74.5% fewer nodes evaluated!

=== Performance Analysis ===
Execution Time (no pruning): 0.00555 seconds
Execution Time (with pruning): 0.00183 seconds
Speedup: 3.03x faster!

=== Theoretical Complexity ===
Plain Minimax Time Complexity: O(b^d)
Alpha–Beta Pruning Time Complexity: O(b^(d/2)) (in best case)
Space Complexity (both): O(b * d) (due to recursion stack)

=== Memory Approximation ===
Estimated memory (no pruning): ~82.45 KB
Estimated memory (with pruning): ~21.04 KB

✅ Observation: Alpha–Beta Pruning explores far fewer nodes, executes faster, and uses less memory — while yielding the same optimal move!
